In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import pandas as pd
from sklearn.preprocessing import StandardScaler
import time
from numpy import random as nr
import math
import warnings
import matplotlib

warnings.filterwarnings("ignore")


# 用于绘制
def plot_results(predicted_data, true_data, picture_name):
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    ax.plot(predicted_data, label='Prediction')
    ax.scatter(np.array(range(len(true_data))),true_data-predicted_data,label='error',marker='.',c='r')  #用于绘制残差
    plt.xlabel('data number')
    plt.ylabel('traffic flow')
    plt.legend()
    plt.savefig(picture_name)
    plt.show()


# 预测性能指标计算
def predict_error(predict_data, true_data):
    # MAPE计算
    MAPE = 0
    MAE = 0
    RMSE = 0
    EC = 0
    predict_q_2, true_q_2 = 0, 0
    for i in range(len(true_data)):
        MAPE = MAPE + abs(predict_data[i] - true_data[i]) / true_data[i]
        MAE = MAE + abs(predict_data[i] - true_data[i])
        RMSE = RMSE + (predict_data[i] - true_data[i]) ** 2
        predict_q_2 = predict_q_2 + predict_data[i] ** 2
        true_q_2 = true_q_2 + true_data[i] ** 2
    MAPE = MAPE / len(true_data)
    MAE = MAE / len(true_data)
    EC = 1 - (RMSE ** 0.5) / (predict_q_2 ** 0.5 + true_q_2 ** 0.5)
    RMSE = (RMSE / len(true_data)) ** 0.5
    print('MAPE:', MAPE)
    print('MAE:', MAE)
    print('EC:', EC)
    print('RMSE:', RMSE)

    return MAPE, MAE, EC, RMSE


def build_model(layers):  # layers[1,50,100,1]
    units = int(layers[1])
    dropout = layers[2]
    layer_num = int(layers[0])

    model = Sequential()

    for i in range(layer_num - 1):
        model.add(LSTM(input_dim=1, output_dim=units, return_sequences=True))
        model.add(Dropout(dropout))

    model.add(LSTM(output_dim=units, return_sequences=False))
    model.add(Dropout(dropout))

    model.add(Dense(output_dim=1))
    model.add(Activation("linear"))

    model.compile(loss='mse', optimizer='rmsprop')

    return model

# 重构数据集结构
# q为延迟阶数，即对未来数据的预测与以前多长时间的交通流数据有关
# s预测步长，即预测未来第几个时刻的交通流数据
def Data_Reconstruct(data, q, s):
    dataX, dataY = [], []  # dataX表示输入变量，dataY表示输出变量

    for i in range(len(Standard_data) - q - s + 1):
        dataX.append(Standard_data[i:(i + q), 0])
        dataY.append(Standard_data[i + q + s - 1, 0])
    return dataX, dataY

#遗传算法函数
#编码
def init_population(pop_size, chromosome_length, dim):
    # 形如[[0,1,..0,1],[0,1,..0,1]...]
    pop = np.array([[[nr.randint(0, 2) for i in chromosome_length] for j in range(pop_size)]for k in range(dim)])
    return pop


# 解码并计算值
def decode_chromosome(pop,pop_size,dim,upper_limit, chromosome_length):
    X = np.zeros(shape=(dim,pop_size))
    temp = np.zeros(shape=(dim,pop_size))
    # print(pop[0,...])
    for i in range(pop.shape[0]):
        # 二进制变成实数，种群中的每个个体对应一个数字
        for j in range(pop.shape[1]):
            for k in range(pop.shape[2]):
                # 就是把二进制转化为十进制的
                temp[i,j]+=pop[i,j,k] * 2 ** k
            X[i,j]=temp[i,j] * upper_limit[i] / (2 ** chromosome_length[i] - 1)
        # 这个是把前面得到的那个十进制的数，再次缩放为另一个实数
        # 注意这个实数范围更广泛，可以是小数了，而前面二进制解码后只能是十进制的数
        # 参考https://blog.csdn.net/robert_chen1988/article/details/79159244  
    return X

def calculate_fitness(layer, Standard_data):
    q = int(layer[3])
    s = int(layer[4])

    # 将数据分为分训练集与测试集
    dataX, dataY = Data_Reconstruct(data=Standard_data, q=q, s=s)  # 重构数据结构
    train_size = int(len(dataX) * 23 / 30)  # 将数据集的前23天用途训练
    test_size = len(dataX) - train_size
    Xtrain_data, Ytrain_data = np.array(dataX[0:train_size]), np.array(dataY[0:train_size])
    Xtest_data, Ytest_data = np.array(dataX[train_size:]), np.array(dataY[train_size:])

    Xtrain_data, Xtest_data = np.reshape(Xtrain_data, (-1, q, 1)), np.reshape(Xtest_data,
                                                                              (-1, q, 1))  # 修改数据输入形式适应LSTM的输入
    Ytest_data = np.reshape(Ytest_data, (-1, 1))

    # 建立模型
    model = build_model(layer)
    # 用训练集训练模型
    model.fit(Xtrain_data, Ytrain_data, batch_size=96, epochs=10, validation_split=0.05)
    # 用模型进行预测
    Ypredict_data = model.predict(Xtest_data)

    # 将数据转化回原数据
    Ypredict_data = Standard.inverse_transform(Ypredict_data)
    Ytest_data = Standard.inverse_transform(Ytest_data)
    Ypredict_data = [np.round(Ypredict_data[i]) for i in range(len(Ypredict_data))]

    # 计算预测性能
    MAPE = predict_error(Ytest_data, Ypredict_data)

    return MAPE, Ytest_data, Ypredict_data

# 计算适应度
def calc_obj_value(pop, pop_size, dim, fitness):
    obj_value = np.zeros(shape=(pop_size, dim))
    X = decode_chromosome(pop, pop_size, dim)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
        # 把缩放过后的那个数，带入我们要求的公式中
        # 种群中个体有几个，就有几个这种“缩放过后的数”
            obj_value[i,j]=fitness[i,j]    #MAPE
            # 这里先返回带入公式计算后的数值列表，作为种群个体优劣的评价
    return obj_value

# 淘汰
def calc_fit_value(obj_value, pop_size, dim):
    fit_value = np.zeros(shape=(pop_size, dim))
    # 去掉小于0的值，更改c_min会改变淘汰的下限
    # 但是如果设置过大，有可能影响了全局最优的搜索
    c_min = 0.25
    for i in range(obj_value.shape[0]):
        for j in range(obj_value.shape[1]):
            if obj_value[i,j] < c_min:
                fit_value[i,j] = obj_value[i,j]
            else:
                fit_value[i,j] = 0.
    # fit_value保存的是活下来的值
    return fit_value


# 找出最优解和最优解的基因编码
def find_best(pop, fit_value, dim, chromosome_length):
    # 用来存最优基因编码
    best_individual = np.zeros(shape=(dim,chromosome_length))
    best_fit=np.zeros(dim)
    # 先假设第一个基因的适应度最好
    for i in range(fit_value.shape[0]):
        best_fit[i] = fit_value[i,0]
    for i in range(len(pop.shape[0])):
        for j in range(len(pop.shape[1])):
            if (fit_value[i,j] > best_fit[i]):
                best_fit[i] = fit_value[i,j]
                best_individual[i] = pop[i,j]
    # best_fit是值
    # best_individual是基因序列
    return best_individual, best_fit


# 计算累计概率
def cum_sum(fit_value):
    # 输入[1, 2, 3, 4, 5]，返回[1,3,6,10,15]
    temp=np.zeros(shape=(fit_value.shape[0],fit_value.shape[1]))
    for i in range(fit_value.shape[0]):
        for j in range(fit_value.shape[1]):
            temp[i,j] = sum(fit_value[i,:j+1])


# 轮赌法选择
def selection(pop, fit_value):
    p_fit_value = np.zeros(shape=(fit_value.shape[0],fit_value.shape[1]))

    # 适应度总和
    for i in range(fit_value.shape[0]):
        total_fit[i] = sum(fit_value[i,:])     
    # 归一化，使概率总和为1
    for i in range(fit_value.shape[0]):
        for j in range(fit_value.shape[1]):
            p_fit_value[i,j]=(fit_value[i,j] / total_fit[i])        
    # 概率求和排序
    cum_sum(p_fit_value)
    # 生成转盘
    ms=np.zeros(shape=(pop.shape[0],pop.shape[1]))
    for i in range(pop.shape[0]):
        ms[i] = sorted([random.random() for j in range(pop.shape[1])])
    fitin = 0
    newin = 0
    for i in range(pop.shape[0]):
        newpop = pop[i,:]
    # 转轮盘选择法
    while newin < pop.shape[1]:
        # 如果这个概率大于随机出来的那个概率，就选这个
        for i in range(pop.shape[0]):
            if (ms[i,newin] < p_fit_value[i,fitin]):
                newpop[i,newin] = pop[i,fitin]
                newin = newin + 1
            else:
                fitin = fitin + 1
    for i in range(pop.shape[0]):
        pop = newpop[i,...]


# 杂交
def crossover(pop, pc):
    # 一定概率杂交，主要是杂交种群种相邻的两个个体
    for i in range(pop.shape[0]):
        for j in range(0,pop.shape[1],2):
            # 随机看看达到杂交概率没
            if (random.random() < pc):
                # 随机选取杂交点，然后交换数组
                cpoint = random.randint(1, pop.shape[2]-1)
                temp1 = np.hstack((pop[i,j,0:cpoint],pop[i,j+1,cpoint:pop.shape[2]]))
                temp2 = np.hstack((pop[i,j+1,0:cpoint],pop[i,j,cpoint:pop.shape[2]]))
                pop[i,j] = temp1
                pop[i,j+1] = temp2


# 基因突变
def mutation(pop, pm):
    # 每条染色体随便选一个变异
    for i in range(pop.shape[0]):
        for j in range(pop.shape[1]):
            if (random.random() < pm):
                mpoint = random.randint(1, pop.shape[1] - 1)
                if (pop[i,j,mpoint] == 1):
                    pop[i,j,mpoint] = 0
                else:
                    pop[i,j,mpoint] = 1

def procedings(record_time, now_time, total_time):
    if float(now_time / total_time) > record_time:
        record_time = record_time + 0.1
        print(str(now_time / total_time * 100) + '%')

# 从文件中读取出数据
dataset = pd.read_csv('data.csv')
raw_data = dataset.iloc[:, 5:6].values

# Standard_data = normalise_windows(raw_data)
# Standard_data = np.array(Standard_data)

# 将数据标准化
Standard = StandardScaler()
Standard_data = Standard.fit_transform(raw_data)

#遗传算法
pop_size = 80  # 种群数量
upper_limit = [20,18,18,6,4]  # 基因中允许出现的最大值
chromosome_length = [20,18,18,6,4]  # 染色体长度
iter = 400
pc = 0.8 # 杂交概率
pm = 0.005  # 变异概率

dim = 5  # 每个个体的维度设为5，分别表示C,ε，γ，q, s
pop = init_population(pop_size, chromosome_length, dim)
best_individual = pop[:,0:,]
X=[]
MAPE=np.zeros(pop_size)
best_position = np.zeros(dim)
best_parameters=np.zeros(dim)
Ypredict_data = np.zeros(shape=(dim,len(raw_data0)))
Ytest_data = np.zeros(shape=(dim,len(raw_data0)))
start_time = time.time()

for i in range(iter):
    best_parameters=decode_chromosome(best_individual, pop_size, dim, upper_limit, chromosome_length)
    fitness, test_data, predict_data = calculate_fitness(best_parameters, Standard_data)
    obj_value = calc_obj_value(pop, pop_size, dim, fitness)  # 个体评价，有负值
    fit_value = calc_fit_value(obj_value, pop_size, dim)  # 个体适应度，不好的归0，可以理解为去掉上面的负值
    best_individual, best_fit = find_best(pop, fit_value, dim, chromosome_length)  # 第一个是最优基因序列, 第二个是对应的最佳个体适度

    selection(pop, fit_value)  # 选择
    crossover(pop, pc)  # 染色体交叉（最优个体之间进行0、1互换）
    mutation(pop, pm)  # 染色体变异（其实就是随机进行0、1取反）
    # 最优解的变化
    if i % 5 == 0:
        for j in range(dim):
            best_position[j]=best_parameters[j,0].copy
            MAPE[j]=best_fit
Ytest_data = test_data
Ypredict_data = predict_data
end_time = time.time()
operate_time = end_time - start_time
print('运行时间:', operate_time)
print("参数：", Leader_pos)
print("MAPE值：", Leader_score)
plot_results(best_Ypredict_data, best_Ytest_data, 'GA-lstm预测图')


#用于绘制数据收敛图
def draw_result(data):
    fig1 = plt.figure(facecolor='white')
    bx = fig1.add_subplot(111)
    bx.plot(data,label='MAPE变化曲线',color='black')
    plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
    plt.xlim(0,10)
    plt.xlabel('搜索次数')
    plt.ylabel('MAPE')
    plt.legend()
    plt.savefig('GA收敛图.png')
    plt.show()

plot_results(best_Ypredict_data,best_Ytest_data,'GA-LSTM预测图.png')

predict_error(best_Ypredict_data,best_Ytest_data)

plot_results(best_Ypredict_data[:96],best_Ytest_data[:96],'GA-LSTM一天预测图.png')